# BERT를 활용해서 Extraction-based MRC 문제를 풀어보기

### Requirements

In [1]:
!pip install datasets==2.7.0 -q
!pip install transformers==4.24.0 -q
!pip install einops

     |████████████████████████████████| 451 kB 32.7 MB/s 
     |████████████████████████████████| 132 kB 78.2 MB/s 
     |████████████████████████████████| 182 kB 80.7 MB/s 
     |████████████████████████████████| 212 kB 74.6 MB/s 
     |████████████████████████████████| 127 kB 79.5 MB/s 
     |████████████████████████████████| 5.5 MB 16.5 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 385 kB/s 


In [2]:
# To use utility functions defined in examples.
!git clone https://github.com/huggingface/transformers.git
import sys
sys.path.append('transformers/examples/question-answering')

Cloning into 'transformers'...
remote: Enumerating objects: 120535, done.
remote: Counting objects: 100% (399/399), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 120535 (delta 198), reused 281 (delta 125), pack-reused 120136
Receiving objects: 100% (120535/120535), 114.25 MiB | 17.34 MiB/s, done.
Resolving deltas: 100% (90003/90003), done.


## 데이터 및 평가 지표 불러오기

In [3]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import load_metric

metric = load_metric('squad')
metric

<ipython-input-4-ab0041441c25>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('squad')


Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

## Pre-trained 모델 불러오기

In [5]:
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [6]:
model_name = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True)

# AutoModelForQuestionAnswering(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [7]:
import torch.nn as nn
from transformers import T5ForConditionalGeneration, AutoModel

class BaselineModel(nn.Module):
    """_summary_
    베이스라인 모델입니다.
    """
    def __init__(self, model_name, num_labels, dropout_rate):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_labels = num_labels
        self.model_name = model_name
        
        if 't5' in self.model_name:
            self.model = T5ForConditionalGeneration.from_pretrained(self.model_name).get_encoder()
        else:
            self.model = AutoModel.from_pretrained(self.model_name)
            
        self.qa_outputs = nn.Sequential(
            nn.Dropout(p=self.dropout_rate),
            nn.Linear(self.model.config.hidden_size, self.num_labels)
        )

    def forward(self, input_ids, token_type_ids, attention_mask, start_positions=None, end_positions=None):
        last_hidden_state = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        logits = self.qa_outputs(last_hidden_state)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        
        return (start_logits, end_logits)

model = BaselineModel(model_name, 2, 0.1)
# model(**batch)

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 설정하기

In [8]:
max_seq_length = 384 # 질문과 컨텍스트, special token을 합한 문자열의 최대 길이
pad_to_max_length = True
doc_stride = 128 # 컨텍스트가 너무 길어서 나눴을 때 오버랩되는 시퀀스 길이
max_train_samples = 16
max_val_samples = 16
preprocessing_num_workers = 4
batch_size = 4
num_train_epochs = 2
n_best_size = 20
max_answer_length = 30

## 전처리하기

Preprocessing code copied from Lecture 1

In [9]:
def prepare_train_features(examples):
    # 주어진 텍스트를 토크나이징 한다. 이 때 텍스트의 길이가 max_seq_length를 넘으면 stride만큼 슬라이딩하며 여러 개로 쪼갬.
    # 즉, 하나의 example에서 일부분이 겹치는 여러 sequence(feature)가 생길 수 있음.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # max_seq_length까지 truncate한다. pair의 두번째 파트(context)만 잘라냄.
        max_length=max_seq_length,
        stride=doc_stride,
        return_token_type_ids=True,
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )
    
    # example 하나가 여러 sequence에 대응하는 경우를 위해 매핑이 필요함.
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # offset_mappings으로 토큰이 원본 context 내 몇번째 글자부터 몇번째 글자까지 해당하는지 알 수 있음.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 정답지를 만들기 위한 리스트
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]
        
        # 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

        # 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [10]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_token_type_ids=True,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples["offset_mapping"]

    tokenized_examples["example_id"] = []
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]
        tokenized_examples["example_id"].append(examples["id"][example_index])
        
        # 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

        # 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

        context_index = 1
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]


    return tokenized_examples

In [11]:
train_data = datasets["train"]
train_data = train_data.select(range(100)) # 몇개 불러올지 지정할 수 있음

column_names = datasets["train"].column_names
train_dataset = train_data.map(
            prepare_train_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
        )
train_dataset.set_format("torch")

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
valid_data = datasets["validation"]
valid_data = valid_data.select(range(52)) # 몇개 불러올지 지정할 수 있음

column_names = datasets["validation"].column_names
valid_dataset = valid_data.map(
            prepare_validation_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )
valid_dataset2 = valid_dataset.remove_columns(["example_id", "offset_mapping"])
valid_dataset2.set_format("torch")

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
from torch.utils.data.dataloader import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size= batch_size, collate_fn=data_collator, pin_memory=True, shuffle=True)
valid_dataloader = DataLoader(valid_dataset2, batch_size= batch_size, collate_fn=data_collator, pin_memory=True, shuffle=True)

In [14]:
from time import time
import torch
import gc
from tqdm.auto import tqdm
import torch.nn.functional as F
import numpy as np
import einops as ein


class BaselineTrainer():
    """
    훈련과정입니다.
    """
    def __init__(self, model, criterion, metric, optimizer, device,
                 train_dataloader, valid_dataloader=None, epochs=1):
        self.model = model
        self.criterion = criterion
        self.metric = metric
        self.optimizer = optimizer
        self.device = device
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.epochs = epochs

        self.is_token_type_ids = False
        check = True
        for model_name in ['roberta', 'distilbert', 'albert', 'camembert', 'flaubert']:
            if model_name in model.model.name_or_path:
                check = False
        if check and 'bert' in model.model.name_or_path:
            self.is_token_type_ids = True

    def train(self):
        """
        train_epoch를 돌고 valid_epoch로 평가합니다.
        """
        for epoch in range(self.epochs):
            self._train_epoch(epoch)
            self._valid_epoch(epoch)
        torch.cuda.empty_cache()
        del self.model, self.train_dataloader, self.valid_dataloader
        gc.collect()
    
    def _train_epoch(self, epoch):
        gc.collect()
        self.model.train()
        epoch_loss = 0
        steps = 0
        pbar = tqdm(self.train_dataloader)
        for i, batch in enumerate(pbar):
            self.optimizer.zero_grad()
            steps += 1

            if self.is_token_type_ids: # BERT 모델일 경우 token_type_ids를 넣어줘야 합니다.
                inputs = {
                    'input_ids': batch['input_ids'].to(self.device),
                    'attention_mask': batch['attention_mask'].to(self.device),
                    'token_type_ids' : batch['token_type_ids'].to(self.device)
                }
            else:
                inputs = {
                    'input_ids': batch['input_ids'].to(self.device),
                    'attention_mask': batch['attention_mask'].to(self.device),
                }

            start_logits, end_logits = self.model(**inputs)

            start_positions = batch['start_positions'].to(self.device)
            end_positions = batch['end_positions'].to(self.device)

            # seq_len 길이만큼 boundary를 설정하여 seq_len 밖으로 벗어날 경우 벗어난 값을 최소값인 0(cls 토큰)으로 설정해줌
            start_positions.clamp(0, start_logits.size(1))
            end_positions.clamp(0, end_logits.size(1))

            # 각 start, end의 loss 평균
            loss = (self.criterion(start_logits, start_positions) + self.criterion(end_logits, end_positions)) / 2
                
            loss.backward()
            epoch_loss += loss.detach().cpu().numpy().item()
            
            self.optimizer.step()
            
            pbar.set_postfix({
                'loss' : epoch_loss / steps,
                'lr' : self.optimizer.param_groups[0]['lr'],
            })
        pbar.close()

    def _valid_epoch(self, epoch):
        val_loss = 0
        val_steps = 0
        start_logits_all, end_logits_all = [], []
        len_val_dataset = self.valid_dataloader.dataset.num_rows
        val_loss_values=[2]
        with torch.no_grad():
            self.model.eval()
            for valid_batch in tqdm(self.valid_dataloader):
                val_steps += 1
                if self.is_token_type_ids: # BERT 모델일 경우 token_type_ids를 넣어줘야 합니다.
                    inputs = {
                        'input_ids': valid_batch['input_ids'].to(self.device),
                        'attention_mask': valid_batch['attention_mask'].to(self.device),
                        'token_type_ids' : valid_batch['token_type_ids'].to(self.device)
                    }
                else:
                    inputs = {
                            'input_ids': valid_batch['input_ids'].to(self.device),
                            'attention_mask': valid_batch['attention_mask'].to(self.device),
                        }

                start_logits, end_logits = self.model(**inputs)

                start_positions = valid_batch['start_positions'].to(self.device)
                end_positions = valid_batch['end_positions'].to(self.device)

                # seq_len 길이만큼 boundary를 설정하여 seq_len 밖으로 벗어날 경우 벗어난 값을 최소값인 0(cls 토큰)으로 설정해줌
                start_positions.clamp(0, start_logits.size(1))
                end_positions.clamp(0, end_logits.size(1))

                loss = (self.criterion(start_logits, start_positions) + self.criterion(end_logits, end_positions)) / 2
                val_loss += loss.detach().cpu().numpy().item()

                start_logits_all.append(start_logits.detach().cpu().numpy())
                end_logits_all.append(end_logits.detach().cpu().numpy())

            start_logits_all = np.concatenate(start_logits_all)[:len_val_dataset]
            end_logits_all = np.concatenate(end_logits_all)[:len_val_dataset]
            metrics = self.metric.compute(start_logits_all, end_logits_all)

            val_loss /= val_steps
            print(f"Epoch [{epoch+1}/{self.epochs}] Val_loss : {val_loss}")
            print(f"Epoch [{epoch+1}/{self.epochs}] Extact Match :", metrics['exact_match'])
            print(f"Epoch [{epoch+1}/{self.epochs}] F1_score :", metrics['f1'])


In [15]:
'''
간단 코드 버전
'''

import collections

class Metrics1():
    def __init__(self, metric, dataset, raw_data, n_best, max_answer_length):
        self.features = dataset # 원본 소스코드에서도 dataset을 의미
        self.examples = raw_data # 원본 소스코드에서는 example을 의미
        self.n_best = n_best
        self.max_answer_length = max_answer_length
        self.metric = metric

    def compute(self, start_logits, end_logits):
        # 각 문서의 id를 키값, index를 밸류값으로 하는 딕셔너리 생성
        example_to_features = collections.defaultdict(list)
        for idx, feature in enumerate(self.features):
            example_to_features[feature["example_id"]].append(idx)

        # 예측 시작
        predicted_answers = []
        for example in tqdm(self.examples):
            example_id = example["id"] # 문서의 id값
            context = example["context"] # 문서의 context
            answers = [] # 정답 text & start_index

            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = self.features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : - self.n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : - self.n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # 컨텍스트에 완전히 포함되지 않는 답변은 생략
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # 길이가 음수거나 max_answer_length를 넘는 답변은 생략
                        if end_index < start_index or end_index - start_index + 1 > self.max_answer_length:
                            continue

                        answer = {
                            "text": context[offsets[start_index][0] : offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                        answers.append(answer)

            if len(answers) > 0:
                best_answer = max(answers, key=lambda x: x["logit_score"])
                predicted_answers.append(
                    {"id": example_id, "prediction_text": best_answer["text"]}
                )
            else:
                predicted_answers.append({"id": example_id, "prediction_text": ""})

        theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in self.examples]
        return self.metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
'''
복잡쓰 코드 버전
'''

import collections

class Metrics2():
    def __init__(self, metric, dataset, raw_data,  n_best, max_answer_length, mode = 'train', version_2_with_negative=None):
        self.features = dataset # 원본 소스코드에서도 전처리가 완료된 dataset을 의미
        self.examples = raw_data # 원본 소스코드에서는 전처리 전의 원본 데이터인 example을 의미
        self.n_best = n_best
        self.max_answer_length = max_answer_length
        self.metric = metric
        self.mode = mode
        self.version_2_with_negative = version_2_with_negative

    def compute(self, all_start_logits, all_end_logits):
        # 각 문서의 id를 키값, index를 밸류값으로 하는 딕셔너리 생성(참조용)
        example_id_to_index = {k: i for i, k in enumerate(self.examples["id"])}

        '''
        키값으로 인덱스, 밸류값으로 동일한 아이디를 가지는 문서들의 index를 가지는 리스트(example_id_to_index 참조)
        ex) features_per_example[defaultdict] : {0: [0], 1: [1], 2: [2], 3: [3, 4], 4: [5, 6]}
        3 : [3,4]인 경우 document_id가 동일하지만 문장의 길이가 max_length보다 길어서 truncation되서 나눠진 데이터
        '''
        features_per_example = collections.defaultdict(list)
        for i, feature in enumerate(self.features):
            features_per_example[example_id_to_index[feature["example_id"]]].append(i)
        
        # prediction, nbest에 해당하는 OrderedDict 생성합니다.
        all_predictions = collections.OrderedDict()
        all_nbest_json = collections.OrderedDict()
        # if self.version_2_with_negative:
        #     scores_diff_json = collections.OrderedDict()

        # Prediction 시작(전체 example들에 대한 main Loop)
        for example_index, example in enumerate(tqdm(self.examples)):
            # 해당하는 현재 example_index(key) ex) 3 : [3,4]에서 feature_indices는 [3,4]에 해당됩니다.
            feature_indices = features_per_example[example_index]

            min_null_prediction = None # minimum null을 담을 공간을 초기화해줍니다.
            prelim_predictions = []
            
            # 현재 example에 대한 모든 feature 생성합니다.
            for feature_index in feature_indices: # ex) [3,4]
                # 각 featureure에 대한 모든 prediction을 가져옵니다.
                start_logits = all_start_logits[feature_index]
                end_logits = all_end_logits[feature_index]

                # logit과 original context의 logit을 mapping합니다.
                offset_mapping = self.features[feature_index]["offset_mapping"]

                # Optional : `token_is_max_context`, 제공되는 경우 현재 기능에서 사용할 수 있는 max context가 없는 answer를 제거합니다
                token_is_max_context = self.features[feature_index].get(
                    "token_is_max_context", None # token_is_max_context가 없다면 None을 반환
                )

                # feature_null_score값이 이전에 truncation된 문장보다 작다면 현재 score값으로 업데이트 해줍니다.
                feature_null_score = start_logits[0] + end_logits[0] # sequence length 중에서 0번 인덱스는 cls vector를 의미
                if (min_null_prediction is None or min_null_prediction["score"] > feature_null_score):
                    min_null_prediction = {
                        "offsets": (0, 0),
                        "score": feature_null_score,
                        "start_logit": start_logits[0],
                        "end_logit": end_logits[0],
                    }

                # n_best size만큼 큰 값 순으로 인덱스 정렬 및 reverse slicing([int:int:-1])
                start_indexes = np.argsort(start_logits)[-1 : (-n_best_size - 1) : -1].tolist()
                end_indexes = np.argsort(end_logits)[-1 : (-n_best_size - 1) : -1].tolist()

                # n_best_size^2 만큼 완전탐색
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # out-of-scope answers는 고려하지 않습니다.
                        if (
                            start_index >= len(offset_mapping) # max_len을 벗어난 경우
                            or end_index >= len(offset_mapping)
                            or offset_mapping[start_index] is None # context가 아닌 question이나 special token일 경우
                            or offset_mapping[end_index] is None
                        ): continue

                        # 길이(end - start)가 < 0 또는 길이가 > max_answer_length(하이퍼 파라미터)인 answer도 고려하지 않습니다.
                        if (
                            end_index < start_index # 길이가 0 미만인 경우
                            or end_index - start_index + 1 > max_answer_length # max_answer_length보다 긴 경우
                        ): continue
                        
                        # 최대 context가 없는 answer도 고려하지 않습니다.
                        if (
                            token_is_max_context is not None # token_is_max_context이 None이라면 if문을 바로 빠져나옵니다.
                            and not token_is_max_context.get(str(start_index), False) # start_index가 포함되어 있지 않다면 반환
                        ): continue

                        # n_best_size내에서 고려할 수 있는 모든 경우를 추가합니다.
                        prelim_predictions.append(
                            {
                                "offsets": (
                                    offset_mapping[start_index][0],
                                    offset_mapping[end_index][1],
                                ),
                                "score": start_logits[start_index] + end_logits[end_index],
                                "start_logit": start_logits[start_index],
                                "end_logit": end_logits[end_index],
                            }
                        )
            # if version_2_with_negative:
            #     # minimum null prediction을 추가합니다.
            #     prelim_predictions.append(min_null_prediction)
            #     null_score = min_null_prediction["score"]

            # feature_indices(ex) [3,4])에 대한 탐색을 끝내고 모든 truncation 문장을 포함해서 가장 좋은 `n_best_size` predictions만 유지합니다.
            predictions = sorted(prelim_predictions,
                                 key=lambda x: x["score"], reverse=True # 내림차순
                                 )[:n_best_size] # n_best_size만큼 남기기
            
            # # 낮은 점수로 인해 제거된 경우 minimum null prediction을 다시 추가합니다.
            # if version_2_with_negative and not any(
            #     p["offsets"] == (0, 0) for p in predictions
            # ): predictions.append(min_null_prediction)

            # offset을 사용하여 original context에서 predict answer text를 수집합니다.
            context = example["context"]
            for pred in predictions:
                offsets = pred.pop("offsets") # offsets key의 value값을 pop  (start, end)
                pred["text"] = context[offsets[0] : offsets[1]] #predictions에 {'text' : predict answer text}를 추가

            # rare edge case에는 null이 아닌 예측이 하나도 없으며 failure를 피하기 위해 fake prediction을 만듭니다.
            if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
                predictions.insert( # 예측에 실패했으므로 가장 높은 점수로써 기본 0값을 넣습니다.
                    0, {"text": "empty",
                        "start_logit": 0.0,
                        "end_logit": 0.0,
                        "score": 0.0}
                )
            # 모든 점수의 소프트맥스를 계산합니다
            scores = np.array([pred.pop("score") for pred in predictions])
            exp_scores = np.exp(scores - np.max(scores))
            probs = exp_scores / exp_scores.sum()

            # 예측값에 확률을 포함합니다.
            for prob, pred in zip(probs, predictions):
                pred["probability"] = prob

            # best prediction을 선택합니다. all_predictions에 id에 해당하는 가장 높은 확률[0]의 예상 text를 추가합니다.
            if not self.version_2_with_negative:
                all_predictions[example["id"]] = predictions[0]["text"]
            # else:
                # # else case : 먼저 비어 있지 않은 최상의 예측을 찾아야 합니다
                # i = 0
                # while predictions[i]["text"] == "":
                #     i += 1
                # best_non_null_pred = predictions[i]

                # # threshold를 사용해서 null prediction을 비교합니다.
                # score_diff = (
                #     null_score
                #     - best_non_null_pred["start_logit"]
                #     - best_non_null_pred["end_logit"]
                # )
                # scores_diff_json[example["id"]] = float(score_diff)  # JSON-serializable 가능
                # if score_diff > null_score_diff_threshold:
                #     all_predictions[example["id"]] = ""
                # else:
                #     all_predictions[example["id"]] = best_non_null_pred["text"]

            # np.float를 다시 float로 casting -> `predictions`은 JSON-serializable 가능
            all_nbest_json[example["id"]] = [
                { k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items() } for pred in predictions
            ]

        # 실제 계산단계
        predicted_answers = [{"id": k, "prediction_text": v} for k, v in all_predictions.items()]
        if self.mode == 'train': # validation일때는 실제 값과 비교하여 계산
            theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in self.examples]
            return self.metric.compute(predictions=predicted_answers, references=theoretical_answers)
        else: # inference일 경우에는 예측값만 반환
            return predicted_answers


In [16]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)
metrics = Metrics2(
    metric = load_metric('squad'),
    dataset = valid_dataset,
    raw_data = valid_data,
    n_best = 20,
    max_answer_length = 30
)
device = torch.device('cuda:0')

trainer = BaselineTrainer(
        model = model.to(device),
        criterion = criterion,
        metric = metrics,
        optimizer = optimizer,
        device = device,
        train_dataloader = train_dataloader,
        valid_dataloader = valid_dataloader,
        epochs=1,
    )

In [17]:
trainer.train()

  0%|          | 0/30 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/16 [00:00<?, ?it/s]

Epoch [1/1] Val_loss : 4.637238919734955


### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

